# Brief

### This is a data set about global and national death information downloaded from who. We will analyze the death information of different countries, different years, different genders and different ages through 8 templates, and generate reports and visualization for reference


# 1. Load package

In [ ]:
%matplotlib inline
import sqlite3
from sqlite3 import Error
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import csv

# 2. Save CSV data into database

In [ ]:
conn = sqlite3.connect("global_death_causes.db")

Summary = pd.read_csv("../7360_iris/data/Summary.csv",dtype = {'country' : str,'cause_id':int,'d_bothsex':float,'d_male':float,'d_female':float,'year':int})

age_death = pd.read_csv("../7360_iris/data/age_death.csv",dtype = {'cause_id' : int,'glo_bothsex':float,'glo_male':float,'glo_female':float,'agegroup':str,'year':int})

country_info = pd.read_csv("../7360_iris/data/country_info.csv",dtype = {'country' : str,'economic_level':str})

death_causes = pd.read_csv("../7360_iris/data/death_causes.csv",dtype={'cause_id':int,'death_cause':str})

population = pd.read_csv("../7360_iris/data/population.csv",dtype = {'country' : str,'sum_pop':float,'male':float,'female':float,'year':int})


In [ ]:
try:
    Summary.to_sql('Summary', conn, if_exists='fail', index=False)

    age_death.to_sql('age_death', conn, if_exists='fail', index=False)

    country_info.to_sql('country_info', conn, if_exists='fail', index=False)

    death_causes.to_sql('death_causes', conn, if_exists='fail', index=False)

    population.to_sql('population', conn, if_exists='fail', index=False)

except:    
    conn.close()

# 3. Create template

In [ ]:
allcause=[]
allcountry=[]
allyear=[]
alllevel=[]
allage=[]
def demo(conn):
    with conn:

            sql = """
                    SELECT distinct country
                    FROM country_info

                    """
            cur = conn.cursor()
            cur.execute(sql)
            result_set = cur.fetchall()
            for i in result_set:
                allcountry.append(i[0])
            
            sql1 = """
                    SELECT distinct year
                    FROM Summary

                    """
            cur = conn.cursor()
            cur.execute(sql1)
            result_set1 = cur.fetchall()
            for i in result_set1:
                allyear.append(i[0])
            
            sql2 = "SELECT death_cause FROM death_causes"
            cur = conn.cursor()
            cur.execute(sql2)
            result_set = cur.fetchall()
            for i in result_set:
                allcause.append(i[0])
            
            sql3='SELECT DISTINCT economic_level FROM country_info'
            cur = conn.cursor()
            cur.execute(sql3)
            result_set = cur.fetchall()
            for i in result_set:
                alllevel.append(i[0])
            
            sql4='SELECT DISTINCT agegroup FROM age_death WHERE agegroup NOTNULL'
            cur = conn.cursor()
            cur.execute(sql4)
            result_set = cur.fetchall()
            for i in result_set:
                allage.append(i[0])
    
def create_connection(db_file):
    """ 
    create a database connection to the SQLite database specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn


def showOptions():
    """ 
    Show the options 
    """
    choices = ["Add data into database", 
			"Delete a cause of death by causer id", 
			"Query the top 5 causes of death global in different years",
			"Query the top five causes of death in different years in a given country", 
			"Query countries with high death rates for a given cause of death",
			"Query trends in the top five causes of death during 2010-2019",
			"Query the top five causes of death based on national income levels",
			"Query the top five causes of death for male based on country and year",
			"Query the top five causes of death for female based on country and year",
			"Query the top five causes of death based on year and age group",
			"exit"]
    print("---------------------------")
    print("Please choose following option:")
    print("1. ", choices[0])
    print("2. ", choices[1])
    print("3. ", choices[2])
    print("4. ", choices[3])
    print("5. ", choices[4])
    print("6. ", choices[5])
    print("7. ", choices[6])
    print("8. ", choices[7])
    print("9. ", choices[8])
    print("10. ", choices[9])
    print("11. ", choices[10]) 

def adddeathcause(conn):
    """
    Insert data into database
    A sample input is: 111, lung cancer
    """
    line = input("Please input the cause_id, death_cause: ")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    with conn:
        sql = "INSERT INTO death_causes VALUES(?, ?)"
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1]))
        conn.commit() 
        print("succeed to add info ", line)
        return cur.lastrowid
    
def deledeathcause(conn):
    """
    Delete a cause of death by causer id
    A sample input is: 20
    """
    line = input("Please input the causer_id to delete: ")
    line = line.strip()
    print(line)
    with conn:
        sql = "DELETE FROM death_causes WHERE cause_id =?"
        cur = conn.cursor()
        cur.execute(sql, [line])
        conn.commit() 
        print("succeed to delete info ", line)
    
def template1(conn):
    """
    Query the top 5 causes of death global in different years.
    """
    print('year includes',allyear)
    year = input("Please input the year to print info:")
    year = year.strip()
    if int(year) not in allyear:
        print("No such result, please enter again.")
    else:
        print('-----------------------------------------')

        with conn:

            sql = """
                    SELECT * 
                    FROM (SELECT death_cause,glo_bothsex
                    FROM death_causes as D,(select cause_id,glo_bothsex
                    FROM age_death 
                    WHERE year = ? AND agegroup="allage"
                    ORDER BY glo_bothsex DESC) as A
                    WHERE D.cause_id = A.cause_id)

                    """
            cur = conn.cursor()
            cur.execute('CREATE TABLE IF NOT EXISTS template1("death_cause"	TEXT,"death_amount"	NUMERIC)')
            cur.execute(sql, [year])
            result_set={}
            result_set = cur.fetchall()
            header = [t[0] for t in cur.description]
            df = pd.DataFrame(list(result_set),columns=header)
            print('Template1 is updated successfully. Here is your report:\n')
            print("""In 2019, the World Health Organization released global health estimates. The new data covers the period from 2000 to 2019 (inclusive). Estimates reveal trends in death caused by diseases and injuries in the past 20 years. It is important to know why people die to improve how people live. Measuring how many people die each year helps to assess the effectiveness of our health systems and direct resources to where they are needed most.
            """)
            print("In",year,',The world’s biggest killer is',result_set[1][0],', responsible for',int(result_set[1][1]/result_set[0][1]*100),
                 '% of the world’s total deaths.',result_set[2][0],'and',result_set[3][0],'are the 2nd and 3rd leading causes of death.',
                 result_set[4][0],'remained the world’s most deadly disease, ranked as the 4th leading cause of death.',
                 result_set[5][0],'are ranked 5th.')
            fig = plt.figure(figsize = (12, 5)) 
            plt.barh(y=(df.death_cause[5],df.death_cause[4],df.death_cause[3],df.death_cause[2],df.death_cause[1]),
                       width=(df.glo_bothsex[5],df.glo_bothsex[4],df.glo_bothsex[3],df.glo_bothsex[2],df.glo_bothsex[1]), height=0.5)
            plt.xlabel('Number of deaths',fontsize=12)
            plt.ylabel('Causes of death',fontsize=12)
            plt.title('TOP 5 causes of death global in '+year+' Year',fontsize=15)
            plt.show()
            cur.execute('CREATE TABLE IF NOT EXISTS template1(death_cause TEXT,glo_bothsex INTEGER)')
            sql = """INSERT INTO template1
                    SELECT * 
                    FROM (SELECT death_cause,glo_bothsex
                    FROM death_causes as D,(select cause_id,glo_bothsex
                    FROM age_death 
                    WHERE year = ? AND agegroup="allage"
                    ORDER BY glo_bothsex DESC) as A
                    WHERE D.cause_id = A.cause_id)

                    """
            cur.execute(sql, [year])
            conn.commit()


    
def template2(conn):
    """
    Query the top five causes of death in different years in a given country.
    """
    print('year includes',allyear)
    print('country includes',allcountry)
    line = input("Please input country, year:")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    if int(values[1]) not in allyear or values[0] not in allcountry:
            print("No such result, please enter again.")
    else:
        print('-----------------------------------------')
        with conn:
            sql ="""
                    SELECT death_cause,country,economic_level,year,d_bothsex
                    FROM death_causes as D,
                    (SELECT *
                    FROM Summary S,country_info C
                    WHERE S.country=C.country AND S.country = ? AND S.year = ?
                    ORDER by d_bothsex DESC) as A
                    WHERE D.cause_id = A.cause_id

                    """
            cur = conn.cursor()
            cur.execute(sql,(values[0],values[1]))
            result_set = cur.fetchall()
            header = [t[0] for t in cur.description]
            df = pd.DataFrame(list(result_set),columns=header)
            print('Template2 is updated successfully. Here is your report:\n')
            print("""The World Bank classifies the world's economies into four income groups – based on gross national income – low, lower-middle, upper-middle and high.
            """)
            print("In",values[1],',the highest cause of death in',values[0],'which is a',result_set[0][2],'country,',
                 result_set[1][0],'is the highest cause of death,which caused',int(result_set[1][4]),'deaths and had ',
                  int(result_set[1][4]/result_set[0][4]*100),'% of total deaths.The next is',result_set[2][0],',the third is',
                  result_set[3][0],',the forth is',result_set[4][0],',and the fifth is',result_set[5][0]+'.')
            fig = plt.figure(figsize = (12, 5)) 
            plt.barh(y=(df.death_cause[5],df.death_cause[4],df.death_cause[3],df.death_cause[2],df.death_cause[1]),
                       width=(df.d_bothsex[5],df.d_bothsex[4],df.d_bothsex[3],df.d_bothsex[2],df.d_bothsex[1]), height=0.5)
            plt.xlabel('Number of deaths',fontsize=12)
            plt.ylabel('Causes of death',fontsize=12)
            plt.title('Top 5 causes of death of '+values[0]+' in '+values[1]+' year',fontsize=15)
            plt.show()
            cur.execute('CREATE TABLE IF NOT EXISTS template2("death_cause" TEXT,"country" TEXT,"level" INTEGER,"year" INTEGER,"amount" INTEGER)')
            sql ="""INSERT INTO template2
                    SELECT death_cause,country,economic_level,year,d_bothsex
                    FROM death_causes as D,
                    (SELECT *
                    FROM Summary S,country_info C
                    WHERE S.country=C.country AND S.country = ? AND S.year = ?
                    ORDER by d_bothsex DESC) as A
                    WHERE D.cause_id = A.cause_id

                    """
            cur.execute(sql,(values[0],values[1]))
            conn.commit()
    
def template3(conn):
    """
    Query countries with high death rates for a given cause of death.
    """
    print('death causes includes',allcause)
    print('year includes',allyear)
    line = input("Please input the death_causes, year: ")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    if values[0] not in allcause or int(values[1]) not in allyear:
        print("No such result, please enter again.")
    else:
        print('-----------------------------------------')
        with conn:
            sql = """
                SELECT country, d_bothsex 
                FROM death_causes natural join Summary NATURAL JOIN country_info NATURAL JOIN population 
                WHERE death_cause=? AND year =? ORDER by d_bothsex / sum_pop DESC LIMIT 5
                """
            cur = conn.cursor()
            cur.execute(sql, (values[0], values[1]))
            result_set = cur.fetchall()
            print('Template3 is updated successfully. Here is your report:\n')
            print("""In 2019, the World Health Organization released global health estimates. The new data covers the period from 2000 to 2019 (inclusive). Estimates reveal trends in death caused by diseases and injuries in the past 20 years. It is important to know why people die to improve how people live. Measuring how many people die each year helps to assess the effectiveness of our health systems and direct resources to where they are needed most.
            """)
            print('In '+str(values[1])+', the country with the highest death rate from '+ str(values[0])+ 'is '+result_set[0][0]+', where '+ str(result_set[0][1])+' died from it. The next is '+ result_set[1][0]+', the third is ' +result_set[2][0]+', the forth is ' + result_set[3][0]+', and the fifth is '+ result_set[4][0]+'.')
            sql1 = """
                SELECT country, d_bothsex / sum_pop as rate 
                FROM death_causes natural join Summary NATURAL JOIN country_info NATURAL JOIN population  
                WHERE death_cause=? AND year =? ORDER by rate
                """
            cur = conn.cursor()
            cur.execute(sql1, (values[0], values[1]))
            result_set1=cur.fetchall()
            header = [t[0] for t in cur.description]
            df = pd.DataFrame(list(result_set1),columns=header)
            fig = plt.figure(figsize = (13, 15)) 
            plt.barh(df['country'],df['rate'],label='death rate')
            plt.xlabel('Death rate',fontsize=12)
            plt.ylabel('Country',fontsize=12)
            plt.title(values[0]+' death rates in countries in '+values[1],fontsize=15)
            plt.show()
            cur.execute('CREATE TABLE IF NOT EXISTS template3("country" TEXT, "d_bothsex" REAL)')
            sql ="""
            INSERT INTO template3
            SELECT country, d_bothsex 
            FROM death_causes natural join Summary NATURAL JOIN country_info NATURAL JOIN population 
            WHERE death_cause=? AND year =? ORDER by d_bothsex / sum_pop DESC LIMIT 5
            """
            cur.execute(sql,(values[0],values[1]))
            conn.commit()

def template4(conn):
    """
    Query the trend of the cause of death during 2010-2019.
    """
    print('death causes includes',allcause)
    print('year includes',allyear)
    line = input("Please input the death_causes, start_year, end_year: ")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    if values[0] not in allcause or int(values[1]) not in allyear or int(values[2]) not in allyear:
        print("No such result, please enter again.")
    else:
        print('-----------------------------------------')
        with conn:
            sql = """
            SELECT death_cause, g1.glo_bothsex, g2.glo_bothsex, g1.year, g2.year, round((g2.glo_bothsex-g1.glo_bothsex)/g1.glo_bothsex,2) as rate
            FROM age_death as g1 LEFT OUTER JOIN age_death as g2 on g1.cause_id=g2.cause_id NATURAL JOIN (SELECT * FROM death_causes WHERE cause_id<>0) 
            WHERE death_cause=? AND g1.year=? AND g2.year=? AND g1.agegroup='allage' AND g2.agegroup='allage'
            """
            cur = conn.cursor()
            cur.execute(sql, (values[0], values[1], values[2]))
            result_set = cur.fetchall()
            if result_set[0][5]>0:
                print('Template4 is updated successfully. Here is your report:\n')
                print("""In 2019, the World Health Organization released global health estimates. The new data covers the period from 2000 to 2019 (inclusive). Estimates reveal trends in death caused by diseases and injuries in the past 20 years. It is important to know why people die to improve how people live. Measuring how many people die each year helps to assess the effectiveness of our health systems and direct resources to where they are needed most.
            """)
                print('Compared with '+ values[1]+', the number of deaths in '+values[0]+' was increased by '+str(result_set[0][5])+' in '+values[2]+'.')
            else:
                print('Template4 is updated successfully. Here is your report:\n')
                print("""
            In 2019, the World Health Organization released global health estimates. The new data covers the period from 2000 to 2019 (inclusive). Estimates reveal trends in death caused by diseases and injuries in the past 20 years. It is important to know why people die to improve how people live. Measuring how many people die each year helps to assess the effectiveness of our health systems and direct resources to where they are needed most.
            """)
                print('Compared with '+ values[1]+', the number of deaths in '+values[0]+' was decreased by '+str(result_set[0][5])+' in '+values[2]+'.')
            cur.execute('CREATE TABLE IF NOT EXISTS template4("death_cause" TEXT, "str_bothsex" REAL NOT NULL, "end_bothsex" REAL NOT NULL, "str_year" REAL NOT NULL, "end_year" REAL NOT NULL, "rate" REAL NOT NULL)')
            sql ="""
            INSERT INTO template4
            SELECT death_cause, g1.glo_bothsex, g2.glo_bothsex, g1.year, g2.year, round((g2.glo_bothsex-g1.glo_bothsex)/g1.glo_bothsex,2) as rate
            FROM age_death as g1 LEFT OUTER JOIN age_death as g2 on g1.cause_id=g2.cause_id NATURAL JOIN (SELECT * FROM death_causes WHERE cause_id<>0) 
            WHERE death_cause=? AND g1.year=? AND g2.year=? AND g1.agegroup='allage' AND g2.agegroup='allage'
            """
            cur.execute(sql,(values[0],values[1]))
            conn.commit()

def template5(conn):
    """
    Query the top five causes of death based on national income levels.
    """
    print('economic levels include',alllevel)
    print('years include',allyear)
    line = input("Please input the economic level, year: ")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    if values[0] not in alllevel or int(values[1]) not in allyear:
        print("No such result, please enter again.")
    else:
        print('-----------------------------------------')
        with conn:
            sql = """
            SELECT  death_cause, sum(d_bothsex)/sum(sum_pop) AS rate
            FROM (SELECT * FROM country_info WHERE economic_level=?) NATURAL JOIN (SELECT * FROM Summary WHERE year=? AND cause_id<>0) NATURAL JOIN death_causes NATURAL JOIN population
            GROUP BY death_cause
            ORDER BY rate DESC LIMIT 5
            """
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1]))
        result_set={}
        result_set = cur.fetchall()
        print('Template5 is updated successfully. Here is your report:\n')
        print("""In 2019, the World Health Organization released global health estimates. The new data covers the period from 2000 to 2019 (inclusive). Estimates reveal trends in death caused by diseases and injuries in the past 20 years. It is important to know why people die to improve how people live. Measuring how many people die each year helps to assess the effectiveness of our health systems and direct resources to where they are needed most.
            """)
        print('The World Bank classifies the world economies into four income groups – based on gross national income – low, lower-middle, upper-middle and high. In '+str(values[1])+', top five causes of death in '+ str(values[0])+' income countries are '+str(result_set[0][0])+', and its death rate is '+ str(round(result_set[0][1],2))+'%. The next is '+ str(result_set[1][0])+', the third is ' +str(result_set[2][0])+', the forth is ' + str(result_set[3][0])+', and the fifth is '+ str(result_set[4][0])+'.')
        header = [t[0] for t in cur.description]
        df = pd.DataFrame(list(result_set),columns=header)
        fig = plt.figure(figsize = (13, 8)) 
        plt.bar(df['death_cause'],df['rate'],label='death rate')
        plt.xlabel('death_cause',fontsize=12)
        plt.ylabel('death rate',fontsize=12)
        plt.title('The top 5 death causes in '+values[0]+' income countries in '+values[1],fontsize=15)
        plt.show()

        cur.execute('CREATE TABLE IF NOT EXISTS template5("death_cause" TEXT, "rate" REAL NOT NULL)')
        sql ="""
            INSERT INTO template5
            SELECT  death_cause, sum(d_bothsex)/sum(sum_pop) AS rate
            FROM (SELECT * FROM country_info WHERE economic_level=?) NATURAL JOIN (SELECT * FROM Summary WHERE year=? AND cause_id<>0) NATURAL JOIN death_causes NATURAL JOIN population
            GROUP BY death_cause
            ORDER BY rate DESC LIMIT 5
            """
        cur.execute(sql,(values[0],values[1]))
        conn.commit()
    

def template6(conn):
    """
    Query the top five causes of death for male based on country and year.
    """
    print('year includes',allyear)
    print('country includes',allcountry)
    line = input("Please input country, year:")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    if int(values[1]) not in allyear or values[0] not in allcountry:
        print("No such result, please enter again.")
    else:
        print('-----------------------------------------')
        with conn:

            sql = """
            SELECT d1.death_cause, s1.d_male 
            FROM Summary as s1, death_causes as d1 
            WHERE s1.cause_id = d1.cause_id AND s1.country = ? AND s1.year = ? 
            ORDER BY s1.d_male DESC LIMIT 6
            """
            cur = conn.cursor()
            cur.execute(sql, (values[0], values[1]))
            result_set={}
            result_set = cur.fetchall()
            print('Template6 is updated successfully. Here is your report:\n')
            print("""In 2019, the World Health Organization released global health estimates. The new data covers the period from 2000 to 2019 (inclusive). Estimates reveal trends in death caused by diseases and injuries in the past 20 years. It is important to know why people die to improve how people live. Measuring how many people die each year helps to assess the effectiveness of our health systems and direct resources to where they are needed most.
            """)
            print('For male, '+'in '+str(values[1])+', the total number of' + result_set[0][0] + ' is ' + str(result_set[0][1]) + ' in ' + str(values[0]) + '. And the cause of death with the highest number is '+ result_set[1][0] +'.Its quantity is ' + str(result_set[1][1])+'. The second most is ' + result_set[2][0]+', and its quantity is ' + str(result_set[2][1])+'.'+ 'The third most is ' + result_set[3][0]+', and its quantity is ' + str(result_set[3][1])+'.The fourth most is ' + result_set[4][0] +', and its quantity is ' + str(result_set[4][1])+'.The fifth most is ' + result_set[5][0]+', and its quantity is ' + str((result_set[5][1]))+'.')
            print("=================================================")
            header = [t[0] for t in cur.description]
            df = pd.DataFrame(list(result_set),columns=header)
            fig = plt.figure(figsize = (13, 8)) 
            plt.bar(df['death_cause'][1:6],df['d_male'][1:6],label='Death population')
            plt.xlabel('death_cause',fontsize=12)
            plt.ylabel('d_male',fontsize=12)
            plt.title('Top five causes of death of male in ' +values[0] +' in '+values[1],fontsize=15)
            plt.show()
        cur.execute('CREATE TABLE IF NOT EXISTS template6("death_cause" TEXT, "d_male" REAL NOT NULL)')
        sql ="""
        INSERT INTO template6
        SELECT d1.death_cause, s1.d_male 
        FROM Summary as s1, death_causes as d1 
        WHERE s1.cause_id = d1.cause_id AND s1.country = ? AND s1.year = ? 
        ORDER BY s1.d_male DESC LIMIT 6
        """
        cur.execute(sql, (values[0], values[1]))
        conn.commit()

              
def template7(conn):
    """
    Query the top five causes of death for female based on country and year.
    """
    print('year includes',allyear)
    print('country includes',allcountry)                                                                  
    line = input("Please input country, year:")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    if int(values[1]) not in allyear or values[0] not in allcountry:
        print("No such result, please enter again.")
    else:
        print('-----------------------------------------')
        with conn:
            sql = """
            SELECT d1.death_cause, s1.d_female 
            FROM Summary as s1, death_causes as d1 
            WHERE s1.cause_id = d1.cause_id AND s1.country = ? AND s1.year = ? 
            ORDER BY s1.d_female DESC LIMIT 6
            """ 
            cur = conn.cursor()
            cur.execute(sql, (values[0], values[1]))
            result_set={}
            result_set = cur.fetchall()
            print('Template7 is updated successfully. Here is your report:\n')
            print("""In 2019, the World Health Organization released global health estimates. The new data covers the period from 2000 to 2019 (inclusive). Estimates reveal trends in death caused by diseases and injuries in the past 20 years. It is important to know why people die to improve how people live. Measuring how many people die each year helps to assess the effectiveness of our health systems and direct resources to where they are needed most.
            """)
            print('For female, '+'in '+str(values[1])+', the total number of' + result_set[0][0] + ' is ' + str(result_set[0][1]) + ' in ' + str(values[0]) + '. And the cause of death with the highest number is '+ result_set[1][0] +'.Its quantity is ' + str(result_set[1][1])+'. The second most is ' + result_set[2][0]+', and its quantity is ' + str(result_set[2][1])+'.'+ 'The third most is ' + result_set[3][0]+', and its quantity is ' + str(result_set[3][1])+'.The fourth most is ' + result_set[4][0] +', and its quantity is ' + str(result_set[4][1])+'.The fifth most is ' + result_set[5][0]+', and its quantity is ' + str((result_set[5][1]))+'.')
            print("=================================================")
            header = [t[0] for t in cur.description]
            df = pd.DataFrame(list(result_set),columns=header)
            fig = plt.figure(figsize = (13, 8)) 
            plt.bar(df['death_cause'][1:6],df['d_female'][1:6],label='Death population')
            plt.xlabel('death_cause',fontsize=12)
            plt.ylabel('d_female',fontsize=12)
            plt.title('Top five causes of death of female in ' +values[0] +' in '+values[1],fontsize=15)
            plt.show()
        cur.execute('CREATE TABLE IF NOT EXISTS template7("death_cause" TEXT, "d_female" REAL NOT NULL)')
        sql ="""
        INSERT INTO template7
        SELECT d1.death_cause, s1.d_female 
        FROM Summary as s1, death_causes as d1 
        WHERE s1.cause_id = d1.cause_id AND s1.country = ? AND s1.year = ? 
        ORDER BY s1.d_female DESC LIMIT 6
        """
        cur.execute(sql, (values[0], values[1]))
        conn.commit()
              
def template8(conn):
    """
    Query the top five causes of death based on year and age group.
    
    """
    print('years include',allyear)
    print('age groups include',allage)                                                                
    line = input("Please input year, agegroup:")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    if int(values[0]) not in allyear or values[1] not in allage:
        print("No such result, please enter again.")
    else:
        print('-----------------------------------------')
    
        with conn:
            sql = """
            SELECT death_cause, glo_bothsex 
            FROM age_death,death_causes 
            WHERE year =? AND agegroup =? AND age_death.cause_id =death_causes.cause_id AND age_death.cause_id != 0 
            ORDER BY glo_bothsex desc limit 5
            """
            cur = conn.cursor()
            cur.execute(sql, (values[0], values[1]))
            result_set = cur.fetchall()
            result = []
            for tup in result_set:
                for x in tup:
                    result.append(x) 
            causes = list()
            deathnub = list()
            for i in result:  
                try:
                    int(i)
                    deathnub.append(i) 
                except:
                    causes.append(i) 
            top5 = ','.join(causes) 
            print('Template8 is updated successfully. Here is your report:\n')
            print("""In 2019, the World Health Organization released global health estimates. The new data covers the period from 2000 to 2019 (inclusive). Estimates reveal trends in death caused by diseases and injuries in the past 20 years. It is important to know why people die to improve how people live. Measuring how many people die each year helps to assess the effectiveness of our health systems and direct resources to where they are needed most.
            """)
            print("In",values[0],'year','agegroup of',values[1],'the leading cause of death is',causes[0],
                  ', which caused',deathnub[0],'deaths, ','the second one is',causes[1],', led to',deathnub[1],
                  'deaths, ','the 3rd and 4th are',causes[2],'and',causes[3],', wtih deaths',deathnub[2],'and',
                  deathnub[3],', the fifth place is',causes[4],', causing',deathnub[4],'.')
            fig = plt.figure(figsize = (13, 8)) 
            plt.bar(causes,deathnub)
            plt.xlabel('death causes',fontsize=12)
            plt.ylabel('Number of deaths',fontsize=12)
            plt.title('TOP 5 death causes of '+values[1]+' in '+values[0]+' Year',fontsize=15)
            plt.show()
        cur.execute('CREATE TABLE IF NOT EXISTS template8(death_cause CHAR NOT NULL,bothsex INTEGER)')
        sql ="""
        INSERT INTO template8 
        SELECT death_cause, glo_bothsex 
        FROM age_death, death_causes 
        WHERE year =? AND agegroup =? AND age_death.cause_id =death_causes.cause_id AND age_death.cause_id != 0 
        ORDER BY glo_bothsex desc limit 5
        """
        cur.execute(sql, (values[0], values[1]))
        conn.commit()

        
"""main"""
database = "global_death_causes.db"
conn = create_connection(database)
demo(conn)
while True:
    showOptions()
    choice = input('Please enter your option: ')
    # with conn:
    if choice == '11':
        conn.close() # close the connection
        break
    elif choice == '1':
        adddeathcause(conn)
    elif choice == '2':
        deledeathcause(conn)
    elif choice == '3':
        template1(conn)
    elif choice == '4':
        template2(conn)      
    elif choice == '5':
        template3(conn)
    elif choice == '6':
        template4(conn)
    elif choice == '7':
        template5(conn)
    elif choice == '8':
        template6(conn)
    elif choice == '9':
        template7(conn)
    elif choice == '10':
        template8(conn)
    else:
        print("This option is not available")
        continue